In [1]:
 # Dependencies
import csv
import matplotlib.pyplot as plt
import requests
import pandas as pd
from sodapy import Socrata
import json
from datetime import date
import datetime
import numpy as np
import folium
import gmaps

## I. Extract and clean monthly CHIP enrollment data per state (6/2017 - 5/2019)

### I.1 Request data using API

In [2]:
## Data query setup
basic_url = "https://data.medicaid.gov/resource/74p5-kixw.json?"

response = requests.get(basic_url).json()
print (len(response))
print(json.dumps(response[0], indent=2, sort_keys=True))

1000
{
  ":@computed_region_7gxu_g3ub": "8915",
  "applications_for_financial_assistance_submitted_to_the_state_based_marketplace": "N/A",
  "final_report": "N",
  "geocoded_column": {
    "coordinates": [
      -93.9196,
      45.7326
    ],
    "type": "Point"
  },
  "individuals_determined_eligible_for_medicaid_at_application": "8347",
  "individuals_determined_eligible_for_medicaid_at_application_footnotes": "Includes CHIP; Count is of Households, Not Individuals",
  "latitude": "45.7326",
  "longitude": "-93.9196",
  "medicaid_and_chip_child_enrollment": "569605",
  "new_applications_submitted_to_medicaid_and_chip_agencies": "18967",
  "preliminary_updated": "P",
  "report_date": "2018-11-01T00:00:00.000",
  "state_abbreviation": "MO",
  "state_expanded_medicaid": "N",
  "state_name": "Missouri",
  "total_applications_for_financial_assistance_submitted_at_state_level": "18967",
  "total_chip_enrollment": "31139",
  "total_medicaid_and_chip_enrollment": "895063",
  "total_medicaid_

In [3]:
# Create dictionary of states
with open('states.csv', mode='r', encoding='utf-8-sig') as infile:
    reader = csv.reader(infile)
    states = {rows[0]:rows[1] for rows in reader}

states_list = [ k for k in states ]
states_list

['AK',
 'AL',
 'AR',
 'AZ',
 'CA',
 'CO',
 'CT',
 'DC',
 'DE',
 'FL',
 'GA',
 'HI',
 'IA',
 'ID',
 'IL',
 'IN',
 'KS',
 'KY',
 'LA',
 'MA',
 'MD',
 'ME',
 'MI',
 'MN',
 'MO',
 'MS',
 'MT',
 'NC',
 'ND',
 'NE',
 'NH',
 'NJ',
 'NM',
 'NV',
 'NY',
 'OH',
 'OK',
 'OR',
 'PA',
 'RI',
 'SC',
 'SD',
 'TN',
 'TX',
 'UT',
 'VA',
 'VT',
 'WA',
 'WI',
 'WV',
 'WY']

In [4]:
# Create list of dates for making the query
dates_list= ['2017-06', '2017-07', '2017-08', '2017-09', '2017-10',
'2017-11', '2017-12', '2018-01', '2018-02', '2018-03',
'2018-04', '2018-05', '2018-06', '2018-07', '2018-08',
'2018-09', '2018-10', '2018-11', '2018-12', '2019-01',
'2019-02', '2019-03', '2019-04', '2019-05']

In [5]:
# Create a dataframe for monthly observations per state 
# for each of the months in the dates list

df = pd.DataFrame(dates_list, columns=['date'])

df['state']=""
df=df.set_index('date')
df.index

df1 = pd.DataFrame(dates_list, columns=['date'])
df1['state']=""
df1=df1.set_index('date')
df1

for i in range(len(states_list)):
    df1['state']=states_list[i]
    df=pd.concat([df, df1])
df.describe
pd.set_option('display.max_rows', None)
df = df.loc[df["state"] != ""]

df.index
df=df.reset_index()
df.head()

,date,state
0,2017-06,AK
1,2017-07,AK
2,2017-08,AK
3,2017-09,AK
4,2017-10,AK


In [6]:
# Add empty columns for adding the following data to the dataframe: 
# coordinates, state name, state abbreviation and month

df['lat']=""
df['lon']=""
df['chip']=""
df['state_name']=""
df['state_abbrev']=""
df['month']=""

df=df.set_index(['date','state'])
df.head()

,,lat,lon,chip,state_name,state_abbrev,month
date,state,,,,,,
2017-06,AK,,,,,,
2017-07,AK,,,,,,
2017-08,AK,,,,,,
2017-09,AK,,,,,,
2017-10,AK,,,,,,


In [7]:
# Make the API request and populate dataframe's empty fields

for index, row in df.iterrows():
    try:
        s = index
        response = requests.get(basic_url+'state_abbreviation='+s[1]+'&report_date='+s[0]+'-01T00:00:00.000').json()
        print(response[0]["state_name"]) # print state name just to see where the loop is
        df.loc[(index[0],index[1]),"month"]=s[0]
        df.loc[(index[0],index[1]),"state_abbrev"]=response[0]["state_abbreviation"]
        df.loc[(index[0],index[1]),"chip"]=response[0]["total_chip_enrollment"]
        df.loc[(index[0],index[1]),"state_name"]=response[0]["state_name"]
        df.loc[(index[0],index[1]),"lat"]=response[0]["latitude"]
        df.loc[(index[0],index[1]),"lon"]=response[0]["longitude"]
    except IndexError as e:
#         df.loc[(index[0],index[1]),"month"]="NaN"
        df.loc[(index[0],index[1]),"state_abbrev"]="NaN"
        df.loc[(index[0],index[1]),"chip"]="NaN"
        df.loc[(index[0],index[1]),"state_name"]="NaN"
        df.loc[(index[0],index[1]),"lat"]="NaN"
        df.loc[(index[0],index[1]),"lon"]="NaN"
        print ('IndexError - "%s"' % str(e) )
    except ValueError as e:
#         df.loc[(index[0],index[1]),"month"]="NaN"
        df.loc[(index[0],index[1]),"state_abbrev"]="NaN"
        df.loc[(index[0],index[1]),"chip"]="NaN"
        df.loc[(index[0],index[1]),"state_name"]="NaN"
        df.loc[(index[0],index[1]),"lat"]="NaN"
        df.loc[(index[0],index[1]),"lon"]="NaN"
        print ('IndexError - "%s"' % str(e) )

Alaska
Alaska
Alaska
Alaska
Alaska
Alaska
Alaska
Alaska
Alaska
Alaska
Alaska
Alaska
Alaska
Alaska
Alaska
Alaska
Alaska
Alaska
Alaska
Alaska
Alaska
Alaska
Alaska
Alaska
Alabama
Alabama
Alabama
Alabama
Alabama
Alabama
Alabama
Alabama
Alabama
Alabama
Alabama
Alabama
Alabama
Alabama
Alabama
Alabama
Alabama
Alabama
Alabama
Alabama
Alabama
Alabama
Alabama
Alabama
Arkansas
Arkansas
Arkansas
Arkansas
Arkansas
Arkansas
Arkansas
Arkansas
Arkansas
Arkansas
Arkansas
Arkansas
Arkansas
Arkansas
Arkansas
Arkansas
Arkansas
Arkansas
Arkansas
Arkansas
Arkansas
Arkansas
Arkansas
Arkansas
Arizona
Arizona
Arizona
Arizona
Arizona
Arizona
Arizona
Arizona
Arizona
Arizona
Arizona
Arizona
Arizona
Arizona
Arizona
Arizona
Arizona
Arizona
Arizona
Arizona
Arizona
Arizona
Arizona
Arizona
California
California
California
California
California
California
California
California
California
California
California
California
California
California
California
California
California
California
California
California
California
C

Ohio
Ohio
Ohio
Oklahoma
Oklahoma
Oklahoma
Oklahoma
Oklahoma
Oklahoma
Oklahoma
Oklahoma
Oklahoma
Oklahoma
Oklahoma
Oklahoma
Oklahoma
Oklahoma
Oklahoma
Oklahoma
Oklahoma
Oklahoma
Oklahoma
Oklahoma
Oklahoma
Oklahoma
Oklahoma
Oklahoma
Oregon
Oregon
Oregon
Oregon
Oregon
Oregon
Oregon
Oregon
Oregon
Oregon
Oregon
Oregon
Oregon
Oregon
Oregon
Oregon
Oregon
Oregon
Oregon
Oregon
Oregon
Oregon
Oregon
Oregon
Pennsylvania
Pennsylvania
Pennsylvania
Pennsylvania
Pennsylvania
Pennsylvania
Pennsylvania
Pennsylvania
Pennsylvania
Pennsylvania
Pennsylvania
Pennsylvania
Pennsylvania
Pennsylvania
Pennsylvania
Pennsylvania
Pennsylvania
Pennsylvania
Pennsylvania
Pennsylvania
Pennsylvania
Pennsylvania
Pennsylvania
Pennsylvania
Rhode Island
Rhode Island
Rhode Island
Rhode Island
Rhode Island
Rhode Island
Rhode Island
Rhode Island
Rhode Island
Rhode Island
Rhode Island
Rhode Island
Rhode Island
Rhode Island
Rhode Island
Rhode Island
Rhode Island
Rhode Island
Rhode Island
Rhode Island
Rhode Island
Rhode Island
Rho

In [8]:
df.head(30)

,,lat,lon,chip,state_name,state_abbrev,month
date,state,,,,,,
2017-06,AK,61.385,-152.2683,12454,Alaska,AK,2017-06
2017-07,AK,61.385,-152.2683,12605,Alaska,AK,2017-07
2017-08,AK,61.385,-152.2683,12763,Alaska,AK,2017-08
2017-09,AK,61.385,-152.2683,12602,Alaska,AK,2017-09
2017-10,AK,61.385,-152.2683,12963,Alaska,AK,2017-10
2017-11,AK,61.385,-152.2683,12802,Alaska,AK,2017-11
2017-12,AK,61.385,-152.2683,12919,Alaska,AK,2017-12
2018-01,AK,61.385,-152.2683,13141,Alaska,AK,2018-01
2018-02,AK,61.385,-152.2683,13163,Alaska,AK,2018-02


### I.2 Clean and manipulate data getting it ready for analyis

In [28]:
# Modify data types
# df=df.reset_index()
df['state_name'] = df['state_name'].astype('str')
df['state_abbrev']=df.state_abbrev.astype(str)
df['date'] = df['date'].astype('datetime64')
df['chip']=df.chip.astype(float)
df['lat']=df.lat.astype(float)
df['lon']=df.lon.astype(float)
df.info() 
# df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1224 entries, 0 to 1223
Data columns (total 10 columns):
level_0         1224 non-null int64
state_name      1224 non-null object
index           1224 non-null int64
date            1224 non-null datetime64[ns]
state           1224 non-null object
lat             1224 non-null float64
lon             1224 non-null float64
chip            1224 non-null float64
state_abbrev    1224 non-null object
month           1224 non-null object
dtypes: datetime64[ns](1), float64(3), int64(2), object(4)
memory usage: 95.7+ KB


In [35]:
# Reset index and set state_name as index
df = df.reset_index()
df=df[['state_name', 'date', 'state', 'lat','lon','chip', 'state_abbrev','month']]
df.head()

,state_name,date,state,lat,lon,chip,state_abbrev,month
0,Alaska,2017-06-01,AK,61.385,-152.2683,12454.0,AK,2017-06
1,Alaska,2017-07-01,AK,61.385,-152.2683,12605.0,AK,2017-07
2,Alaska,2017-08-01,AK,61.385,-152.2683,12763.0,AK,2017-08
3,Alaska,2017-09-01,AK,61.385,-152.2683,12602.0,AK,2017-09
4,Alaska,2017-10-01,AK,61.385,-152.2683,12963.0,AK,2017-10


In [37]:
# Add a colummn to df with the CHIP enrollment initial value (2017-06) per state


chip_initial=df[['month', 'state_name', 'chip']]
chip_initial =chip_initial.loc[chip_initial["month"] == "2017-06",:]
chip_initial.head()

chip_change = pd.merge(df, chip_initial, on="state_name")
chip_change.head()

,state_name,date,state,lat,lon,chip_x,state_abbrev,month_x,month_y,chip_y
0,Alaska,2017-06-01,AK,61.385,-152.2683,12454.0,AK,2017-06,2017-06,12454.0
1,Alaska,2017-07-01,AK,61.385,-152.2683,12605.0,AK,2017-07,2017-06,12454.0
2,Alaska,2017-08-01,AK,61.385,-152.2683,12763.0,AK,2017-08,2017-06,12454.0
3,Alaska,2017-09-01,AK,61.385,-152.2683,12602.0,AK,2017-09,2017-06,12454.0
4,Alaska,2017-10-01,AK,61.385,-152.2683,12963.0,AK,2017-10,2017-06,12454.0


In [45]:
# Calculate the % change in enrollment by month 
# where the comparison month is always 6/2017 

# df = df.set_index(['state_name'])
chip_change['chip_change'] = (chip_change['chip_x']/chip_change['chip_y'])-1
chip_change['chip_change'] = pd.Series(["{0:.2f}%".format(val * 100) for val in chip_change['chip_change']])


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1224 entries, 0 to 1223
Data columns (total 11 columns):
state_name      1224 non-null object
date            1224 non-null datetime64[ns]
state           1224 non-null object
lat             1224 non-null float64
lon             1224 non-null float64
chip_x          1224 non-null float64
state_abbrev    1224 non-null object
month_x         1224 non-null object
month_y         1224 non-null object
chip_y          1224 non-null float64
chip_change     1224 non-null float64
dtypes: datetime64[ns](1), float64(5), object(5)
memory usage: 114.8+ KB


In [48]:
chip_change.head()

,state_name,date,state,lat,lon,chip_x,state_abbrev,month_x,month_y,chip_y,chip_change
0,Alaska,2017-06-01,AK,61.385,-152.2683,12454.0,AK,2017-06,2017-06,12454.0,0.00%
1,Alaska,2017-07-01,AK,61.385,-152.2683,12605.0,AK,2017-07,2017-06,12454.0,1.21%
2,Alaska,2017-08-01,AK,61.385,-152.2683,12763.0,AK,2017-08,2017-06,12454.0,2.48%
3,Alaska,2017-09-01,AK,61.385,-152.2683,12602.0,AK,2017-09,2017-06,12454.0,1.19%
4,Alaska,2017-10-01,AK,61.385,-152.2683,12963.0,AK,2017-10,2017-06,12454.0,4.09%


In [50]:
# Keep only relevant columns
chip_change=chip_change[['state_name', 'date', 'lat','lon', 'state_abbrev','month_y','chip_x', 'chip_change']]
chip_change.head()

,state_name,date,lat,lon,state_abbrev,month_y,chip_x,chip_change
0,Alaska,2017-06-01,61.385,-152.2683,AK,2017-06,12454.0,0.00%
1,Alaska,2017-07-01,61.385,-152.2683,AK,2017-06,12605.0,1.21%
2,Alaska,2017-08-01,61.385,-152.2683,AK,2017-06,12763.0,2.48%
3,Alaska,2017-09-01,61.385,-152.2683,AK,2017-06,12602.0,1.19%
4,Alaska,2017-10-01,61.385,-152.2683,AK,2017-06,12963.0,4.09%


In [51]:
# Rename columns
chip_change.rename(columns={'state_name':'state',
                          'chip_x':'chip_enrollment',
                          'month_y':'month'}) 

,state,date,lat,lon,state_abbrev,month,chip_enrollment,chip_change
0,Alaska,2017-06-01,61.3850,-152.2683,AK,2017-06,12454.0,0.00%
1,Alaska,2017-07-01,61.3850,-152.2683,AK,2017-06,12605.0,1.21%
2,Alaska,2017-08-01,61.3850,-152.2683,AK,2017-06,12763.0,2.48%
3,Alaska,2017-09-01,61.3850,-152.2683,AK,2017-06,12602.0,1.19%
4,Alaska,2017-10-01,61.3850,-152.2683,AK,2017-06,12963.0,4.09%
5,Alaska,2017-11-01,61.3850,-152.2683,AK,2017-06,12802.0,2.79%
6,Alaska,2017-12-01,61.3850,-152.2683,AK,2017-06,12919.0,3.73%
7,Alaska,2018-01-01,61.3850,-152.2683,AK,2017-06,13141.0,5.52%
8,Alaska,2018-02-01,61.3850,-152.2683,AK,2017-06,13163.0,5.69%
9,Alaska,2018-03-01,61.3850,-152.2683,AK,2017-06,13280.0,6.63%
